# Gentle introduction to Bayesian calibration, validation, and prediction with application to Covid-19 data
---

Prashant K. Jha and J. Tinsley Oden

Oden Institute for Computational Engineering and Sciences

University of Texas at Austin, TX


> ## Abstract
>
> We apply the Bayesian techniques of the model calibration, validation, and prediction to the real world data. We consider the confirmed cases of Covid-19 in Japan as the data and calibrate and validate the generalized (sub-exponential) growth model. We use the calibrated model to make a prediction at 100$^{\text{th}}$ day and quantify the uncertainty in the prediction.

# Introduction
Consider a transient scalar field $C: [0,T] \to [0, \infty)$ denoting the number of confirmed cases of Covid-19 virus as a function of time (in units of days) in some specific region, say Japan. We consider a simple generalized growth model for $C$. Following [5], we assume that $C$ satisfies following ODE:

\begin{align*}
\frac{d C}{dt} = rC(t)^p, \qquad \forall t \in (0,T], \hspace{40pt}(1)
\end{align*}

where $r\geq0$ is the growth rate and $p\in [0,1]$ is the deceleration of growth. When $p=0$, the model is a linear growth model, and when $p=1$, the model is an exponential growth model. Given the initial condition, i.e. the number of Covid-19 cases at $t=0$, $C_0$, (1) can be solved to get, for all $t\in [0,T]$, 

\begin{align*}
C(t) = \left( \frac{r}{m}t + (C_0)^{1/m} \right)^m, \hspace{40pt}(2)
\end{align*}

when $p \in (0,1)$. When $p=0$, $C(t) = C_0 + rt$, and when $p=1$, $C(t) = C_0 \exp[rt]$.
Model, for different parameters $(r,p)$, is depicted below:

![](results/Japan_model.png)

Objective of this work is to introduce the Bayesian techniques of model calibration, validation, and prediction by applying it to the present day relevant problem. We consider Japan's Covid-19 cases as the given data to fit the model (2) using Bayesian approach. We highlight and discuss various steps in the Bayesian approach including the practical challenges occurring during the numerical implementation. 

We seek to find the optimal value of parameters $(r,p)$ in the model (2) that best describes the real Covid-19 data. In other words, we want to fit (2) to the Covid-19 data. In the Bayesian approach, instead of finding one fixed value of parameter $(r,p)$, we seek the joint probability distribution of $(r,p)$. Using the joint probability distribution, we not only make the prediction about the quantity of interest (QoI), we can also measure the uncertainty in the prediction (or confidence in the prediction). In the remainder of this section, we present a brief overview of Bayesian learning in the context of fitting model (2) to the Covid-19 data. Interested readers can find more details in the reference [4] and references therein.

Let $\theta = (r,p) \in \Theta := [0,\infty) \times [0,1]$ denote the model parameter where $\Theta$ is the space where the parameters live in. Let $I = [0,T]$ be the time interval of interest, where $T$ is the final time at which we want to get the model prediction. We denote the discrete data $Y = (Y_1, Y_2, .., Y_N)$ where $Y_i$ is the total number of Covid-19 cases in specific region at day $i$. Let $C(t; \theta)$ denote the model output, i.e. the total number of confirmed cases, at time $t$ for parameter $\theta$. 

Suppose $\pi(\theta)$ denotes the prior probability distribution of the model parameter, $\pi(Y|\theta)$ denotes the conditional probability of the data when the parameter is fixed to $\theta$ (also called the likelihood function), $\pi(\theta|Y)$ denotes the conditional probability of the parameter for a given data $Y$ (also called the posterior), and $\pi(Y)$ denotes the evidence. We apply the Bayes' rule to relate the posterior (unknown) to the likelihood (known) and the prior (assumed):

\begin{align*}
\pi(\theta |Y) = \frac{\pi(Y|\theta)\pi(\theta)}{\pi(Y)}, \hspace{40pt}(3)
\end{align*}

where evidence $\pi(Y)$ is the marginalization of the numerator in (3) (so that the posterior is integrated to $1$). It is given by

\begin{align*}
\pi(Y) = \int_{\Theta} \pi(Y|\theta) \pi(\theta) d\theta. \hspace{40pt}(4)
\end{align*}

We see that, up to a proportionality constant, posterior is proportional to the multiple of likelihood function and prior distribution. Once the definition of the prior and the likelihood function is fixed, we can generate a parameter samples according to the posterior distribution using one of the Markov chain Monte Carlo methods (more details below).

## Prior distribution
We assume no prior knowledge about the parameters $(r,p)$. Therefore, we define the prior distribution as a uniform distribution over $\Theta$, i.e.,

\begin{align*}
\pi(\theta) = \chi_{\Theta}(\theta), \hspace{40pt}(5)
\end{align*}

where $\chi_{\Theta}$ is the usual indicator function.

## Noise in the data and the model inadequacy

### Experimental noise
The data $Y$ can be noisy, that is, the real number of Covid-19 cases could be bit different from the recorded data $Y$. This is called the experimental noise. Suppose $g$ is the real data, $Y$ is the recorded data with some margin of error, and $\epsilon$ is the noise, then $Y$ must be related to $g$ by some function $f$ (unknown)

\begin{align*}
Y = f(g, \epsilon).
\end{align*}

To proceed further, we need to assume some reasonable form of this function $f$. We suppose that $\epsilon$ follows the Gaussian distribution with mean $0$, take $f(g,\epsilon) = g + \epsilon$ (additive noise). This results in 

\begin{align*}
Y = g + \epsilon. \hspace{40pt}(6)
\end{align*}

Since $Y = (Y_1, ..., Y_N)$ is a vector, we can assume $\epsilon = (\epsilon_1, .., \epsilon_N)$ where each $\epsilon_i$ is given by the Gaussian distribution with 0 mean and $\sigma_{i}$ std deviation.

### Model inadequacy
For fixed parameter $\theta$, the model output at $t_i$ is denoted as $C(t_i; \theta)$. Let $\bar{C}(\theta) := (C(t_i;\theta), ..., C(t_N;\theta))$ is the vector of model output. Assuming that the model is imperfect, we introduce an additive modeling error $\gamma(\theta)$ such that the real data and the model output $\bar{C}(\theta)$ are related to each other by:

\begin{align*}
g - \bar{C}(\theta) = \gamma(\theta).  \hspace{40pt}(7)
\end{align*}

$\gamma$ can be assumed to follow some simpler probability distribution. As we see next, we can combine the experimental noise and the model inadequacy, and assume the probability distribution for the combined error $\gamma + \epsilon$.

Following [Section 4.3, 4], and also see (6), we have

\begin{align*}
Y - \epsilon - \bar{C}(\theta) = \gamma(\theta) \Rightarrow Y - \bar{C}(\theta) = \epsilon + \gamma(\theta), \hspace{40pt}(8)
\end{align*}

i.e., the difference between the recorded data and the model output is equal to the sum of the noise and the model inadequacy. 

In rest of this article, we assume $\epsilon + \gamma(\theta) \sim {\cal{N}}(\boldsymbol{0}, \boldsymbol{\sigma})$, where $\boldsymbol{\sigma}$ is the $N\times N$ diagonal matrix with $\boldsymbol{\sigma}_{ii} = \sigma$ for $1\leq i \leq N$. Here $\theta\sim {\cal{N}}(\mu, \sigma)$ means that $\theta$ is sampled from Gaussian distribution or the $\theta$ is the random variable with the probability distribution given by ${\cal{N}}(\mu, \sigma)$.


## Likelihood function
Recall that $\bar{C}(\theta) = (C(t_1; \theta),..., C(t_N; \theta))$ is a vector of model output at times $(t_1, ..., t_N)$ associated to the data $Y$. We define the likelihood function as follows:

\begin{align*}
\pi(Y|\theta) &= {\cal{N}}(Y - \bar{C}(\theta), \boldsymbol{\sigma}) \notag \\
&= \prod_{i=1}^{N} \frac{1}{\sigma_{ii}\sqrt{2\pi}} \exp\left[ -\frac{|Y_i - C(t_i; \theta)|^2}{2\sigma_{ii}^2} \right], \hspace{40pt}(9)
\end{align*}

where we have assumed $\boldsymbol{\sigma}$ to be the diagonal matrix with $\sigma_{ii} = \sigma$ for $1\leq i\leq N$. We can also compute the logarithm of the likelihood function:

\begin{align*}
  \log(\pi(Y|\theta)) = \sum_{i=1}^N \left[ -\frac{|Y_i - C(t_i; \theta)|^2}{2\sigma_{ii}^2} -\log(\sigma_{ii}\sqrt{2\pi})  \right]. \hspace{40pt}(10)
\end{align*}

## Posterior distribution
With the definitions of the likelihood function and the prior distribution at hand, we apply the Bayes' rule to get the posterior distribution:

\begin{align*}
  \pi(\theta|Y) \propto \pi(Y|\theta) \pi(\theta). \hspace{40pt}(11)
\end{align*}

One can change the proportionality sign to equality by introducing the evidence $\pi(Y)$, see (3). 

## Quantity of interest
Once the model is calibrated and validated, we use the posterior distribution for the model parameters to compute the quantity of interest (QoI). Suppose $Q(t, C)$ is the quantity of interest which is a function of time $t$ and function of model output $C$. With the posterior $\pi(\theta|Y)$, the quantity of interest $Q$ in the Bayesian approach is a random field as

\begin{align*}
  Q = Q(t, C) = Q(t, C(\theta)) \hspace{40pt}(12)
\end{align*}

is a function of $\theta$ through model $C$, and $\theta$ is the random field with the probability distribution $\pi(\theta|Y)$. The mean of QoI is given by

\begin{align*}
  E[Q](t, C) = \int_{\Theta} Q(t, C(\theta)) \pi(\theta|Y) d\theta. \hspace{40pt}(13)
\end{align*}

The uncertainty in the predicted QoI is given by the standard deviation of $Q$, i.e.,

\begin{align*}
  V[Q](t, C) = \int_{\Theta} (Q(t, C(\theta)) - E[Q](t, C))^2 \pi(\theta|Y) d\theta. \hspace{40pt}(14)
\end{align*}


## Markov chain Monte Carlo simulation
While (11) relates the known distributions, the likelihood function and the prior distribution, to the desired posterior distribution, we still need a method to find the posterior distribution using (11). 

Let $f$ and $g$ are two probability distributions such that $f\propto g$. The idea is to use the known distribution $g$ to compute the target distribution $f$. This is achieved by methods collectively known as Markov chain Monte Carlo. In this work, we use the Metropolis-Hastings method [1]. We consider a symmetric proposal distribution $q(\theta^\ast|\theta) \sim {\cal{N}}(\theta, \boldsymbol{\sigma}_{p})$ where $\theta^\ast$ is the new sample point, $\theta$ is the current sample point, and $\boldsymbol{\sigma}_p$ is the covariance matrix. From (11), we have a target distribution $f = \pi(\theta|Y)$ and the known probability distribution $g = \pi(Y|\theta) \pi(\theta)$. The acceptance distribution $\alpha(\theta^\ast, \theta)$, which gives the probability of accepting sample $\theta^\ast$ given that we are at sample $\theta$, is defined as:

\begin{align*}
\alpha(\theta^\ast, \theta) = \frac{q(\theta| \theta^\ast) g(\theta^\ast)}{q(\theta^\ast | \theta) g(\theta)} = \frac{g(\theta^\ast)}{g(\theta)}, \hspace{40pt}(15)
\end{align*}

where we used the fact that $q$ is symmetric and so $q(x|y) = q(y|x)$. If $\alpha \geq 1$ then we accept the sample $\theta^\ast$ and if $\alpha < 1$ then we reject the sample $\theta^\ast$ with probability $\alpha$. The MH (Metropolis-Hastings) is implemented in function `metropolis_hastings`.

## Posterior distribution from the MCMC samples
One last practical aspect of the Bayesian approach is how one can sample from the accepted samples of MCMC method or in other words what is the probability distribution inherent in the accepted MCMC samples? This question arises when we go from the calibration step to the validation step. There are two methods to obtain the probability distribution from the MCMC samples:

- *Gaussian approximation:* Compute the mean $\mu_{sample}$ and the standard deviation $\sigma_{sample}$ of the samples and approximate the posterior by Gaussian distribution, $\pi(\theta|Y) \sim {\cal{N}}(\mu_{sample}, \sigma_{sample})$. 

- *Kernel density estimation:* A more accurate approach is to apply the kernel density estimation to approximate the posterior. 

We will compare the results with above two approximations in Sec. 4.

## Calibration, validation, and prediction
Bayesian fitting of the data involves three steps:

### Calibration
We divide the data in the calibration set and the validation set. Let $Y^{c}, \bar{C}^c(\theta), \pi^c(\theta), \pi^c(Y^c|\theta), \pi^c(\theta|Y^c)$ denote the calibration data, model output at times associated to calibration data, prior in the calibration step, likelihood function defined on the calibration data $Y^c$ and model output $\bar{C}^c$, and calibration posterior. The calibration of the model consists of three steps:

[1.] Select the prior as follows
    
\begin{align*}
\pi^c(\theta) = \chi_{\Theta}(\theta).
\end{align*}
    
The likelihood $\pi^c(Y^c|\theta)$ is defined according to (9) with the data $Y^c$ instead of $Y$. The posterior follows the Bayes' rule, i.e., $\pi^c(\theta|Y^c)\propto \pi^c(Y^c|\theta)\pi^c(\theta)$. 
    
[2.] Apply MCMC to get the parameter samples following $\pi^c(\theta|Y^c)$ distribution.
    
[3.] Find the approximate posterior distribution $\pi^c(\theta|Y^c)$ from the MCMC samples, using either the Gaussian approximation or the Kernel Density Estimation. 
  
### Validation 
Let $Y^v, \bar{C}^v(\theta), \pi^v(\theta), \pi^v(Y^v|\theta), \pi^v(\theta|Y^v)$ are the validation data, model output at times corresponding to validation data, prior in the validation step, likelihood function defined on the validation data, and the validation posterior. The steps are as follows:
  
[1.] Select prior as follows
    
\begin{align*}
\pi^v(\theta) = \pi^c(\theta|Y^c).
\end{align*}
    
The likelihood $\pi^v(Y^v|\theta)$ is defined similar to $\pi^c(Y^c|\theta)$. The posterior satisfies $\pi(\theta|Y^v)\propto \pi^v(Y^v|\theta)\pi^v(\theta)$. 
    
[2.] Apply MCMC to get the parameter samples following $\pi^v(\theta|Y^v)$ distribution. Suppose the set of samples is denoted as $\Theta^v = (\theta^v_1, ..., \theta^v_{N^v})$.
    
[3.] Compute the validation error. Quantity of interest is the number of confirmed cases at validation times $(t^v_1, ..., t^v_{N_{valid}})$. At any given time, the number of confirmed cases is the mean of model ouput over MCMC parameter samples, i.e.
    
\begin{align*}
  Q^v(t, C) := \frac{1}{N^{v}} \sum_{s=1}^{N^{v}} C(t; \theta^v_{s}). \hspace{40pt}(16)
\end{align*}
    
We define the validation error $e^v$ as follows:

\begin{align*}
  e^v := \frac{\sum_{i=1}^{N_{valid}} |Y^v_i - Q^v(t^v_i, C)|^2 }{\sum_{i=1}^{N_{valid}} |Y^v_i|^2}. \hspace{40pt}(17)
\end{align*}

We consider the model as "Not invalid" if $e^v \leq \gamma_{tol}$. $\gamma_{tol}$ is the user defined tolerance.
  
### Prediction
If the model is "Not invalid", then the number of confirmed cases $Q = C(T=100; \theta)$ is the random field. The mean of $Q$ is the model prediction and the standard deviation is the uncertainty in the prediction. 

### Citing this work
For citations, use following bibliographic referece

>J. Tinsley Oden, Prashant K. Jha, Lianghao Cao, Taemin Heo, Jing Hu, Mathew Hu, Jonathan Kelley, Jaime D. Mora Paz, Cyrus Neary, Akhil Potla, Sheroze Sherriffdeen, Chase Tessmer, and Christine Yang, "Assessment of Predictability of a Class of Models of Growth of Coronavirus 19 Cases as an Exercise in Predictive Computational Science ," Oden Institute REPORT 20-10, Oden Institute for Computational Engineering and Sciences, The University of Texas at Austin, May 2020.